In [ ]:
# ==================== CLASSIFICATION REPORT & CONFUSION MATRIX ====================
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from transformers import ViTImageProcessor, ViTForImageClassification
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ==================== SETUP ====================
# Initialize processor
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

def vit_transformers(image):
    inputs = processor(images=image, return_tensors="pt")
    return inputs['pixel_values'].squeeze(0)

# Load test dataset
test_dataset = ImageFolder(root=r"C:\Users\Ahmed Pasha\OneDrive\Desktop\garbage\data\test",transform=vit_transformers)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class_names = test_dataset.classes
print(f"Classes: {class_names}")
print(f"Number of test images: {len(test_dataset)}")

# Load model
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224",num_labels=len(class_names),ignore_mismatched_sizes=True)
model.load_state_dict(torch.load(r"C:\Users\Ahmed Pasha\OneDrive\Desktop\garbage\model\best_vit_model.pth", map_location=device))
model.to(device)
model.eval()
print("✅ Model loaded successfully!")

# ==================== GET PREDICTIONS ====================
def get_predictions(model, loader):
    """Get all predictions and true labels"""
    all_predictions = []
    all_labels = []
    all_probabilities = []
    
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(pixel_values=images)
            logits = outputs.logits
            probabilities = torch.nn.functional.softmax(logits, dim=1)
            predictions = torch.argmax(logits, dim=1)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
    
    return np.array(all_predictions), np.array(all_labels), np.array(all_probabilities)

print("\n🔄 Getting predictions...")
predictions, true_labels, probabilities = get_predictions(model, test_loader)
print("✅ Predictions obtained!")

# ==================== CLASSIFICATION REPORT ====================
print("\n" + "="*80)
print("CLASSIFICATION REPORT")
print("="*80)
report = classification_report(
    true_labels, 
    predictions, 
    target_names=class_names,
    digits=4
)
print(report)

# Save classification report
report_dict = classification_report(
    true_labels, 
    predictions, 
    target_names=class_names,
    output_dict=True
)
df_report = pd.DataFrame(report_dict).transpose()
df_report.to_csv("classification_report.csv")
print("✅ Classification report saved to 'classification_report.csv'")

# ==================== CONFUSION MATRIX ====================
print("\n🔄 Creating confusion matrix...")
cm = confusion_matrix(true_labels, predictions)

# Plot confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=class_names,
    yticklabels=class_names,
    cbar_kws={'label': 'Count'}
)
plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Confusion matrix saved to 'confusion_matrix.png'")

# ==================== NORMALIZED CONFUSION MATRIX ====================
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(12, 10))
sns.heatmap(
    cm_normalized, 
    annot=True, 
    fmt='.2%', 
    cmap='RdYlGn',
    xticklabels=class_names,
    yticklabels=class_names,
    cbar_kws={'label': 'Percentage'}
)
plt.title('Normalized Confusion Matrix (Percentages)', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('confusion_matrix_normalized.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Normalized confusion matrix saved to 'confusion_matrix_normalized.png'")

# ==================== PER-CLASS ACCURACY ====================
print("\n" + "="*80)
print("PER-CLASS ACCURACY")
print("="*80)
per_class_accuracy = cm.diagonal() / cm.sum(axis=1)
for i, class_name in enumerate(class_names):
    print(f"{class_name:.<30} {per_class_accuracy[i]*100:.2f}%")

# Plot per-class accuracy
plt.figure(figsize=(10, 6))
plt.bar(class_names, per_class_accuracy * 100, color='steelblue', edgecolor='black')
plt.xlabel('Class', fontsize=12)
plt.ylabel('Accuracy (%)', fontsize=12)
plt.title('Per-Class Accuracy', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.ylim([0, 105])
for i, v in enumerate(per_class_accuracy * 100):
    plt.text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom')
plt.tight_layout()
plt.savefig('per_class_accuracy.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Per-class accuracy plot saved to 'per_class_accuracy.png'")

c:\Users\Ahmed Pasha\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


c:\Users\Ahmed Pasha\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Classes: ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']
Number of test images: 1464


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FileNotFoundError: [Errno 2] No such file or directory: '../models/best_vit_model.pth'